In [1]:
%load_ext autoreload
%autoreload 2

from typing import List, Tuple
import torch

import pytorch_lightning as pl
from src.models.model import RsnaTimmModel
from src.data.datamodule import RsnaDataModule

from pytorch_lightning.utilities.seed import seed_everything
seed_everything(44, True)

/home/jovyan/.imgenv-eoastafurov-0/lib/python3.7/site-packages/pytorch_lightning/utilities/seed.py:49: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
  "`pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be"
Global seed set to 44


44

In [2]:
model = RsnaTimmModel.load_from_checkpoint('/home/jovyan/rsna/greedy_soup.ckpt')

print('loaded!')
conf = model.conf
conf['img_folder_inference'] = '/home/jovyan/Datasets/rsna/test_images_1024_dali'
conf["batch_size"] = 32

dm = RsnaDataModule(conf)
dm.prepare_data()
dm.setup()

Global seed set to 44
/home/user/conda/lib/python3.7/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
Global seed set to 44


loaded!
0.0212056121749463 0.02101608187134503


/home/jovyan/rsna/src/data/dataset.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  + ".png"


In [3]:
trainer = pl.Trainer(accelerator='gpu', devices=1, precision=16)
predictions = trainer.predict(model, dm)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


0.0212056121749463 0.02101608187134503


/home/jovyan/rsna/src/data/dataset.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  + ".png"
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [4]:
trainer.validate(model, dm)
#===========================================================================#
            # = VALID LOSS: 2.076
            # = VALID cF1: 0.074
            # = VALID cF1_thresh 0.486, thr value = 0.846
#===========================================================================#

0.0212056121749463 0.02101608187134503


/home/jovyan/rsna/src/data/dataset.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  + ".png"
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

        #===========================================================================#
            = VALID LOSS: 1.16
            = VALID cF1: 0.062
            = VALID cF1_thresh 0.155, thr value = 0.842
        #===========================================================================#


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_cf1_epoch       │   0.053131841123104095    │
│   val_cf1_thresh_epoch    │    0.1453939825296402     │
│      val_loss_epoch       │    1.1604751348495483     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss_epoch': 1.1604751348495483,
  'val_cf1_epoch': 0.053131841123104095,
  'val_cf1_thresh_epoch': 0.1453939825296402}]

In [ ]:
        # #===========================================================================#
        #     = VALID LOSS: 1.149
        #     = VALID cF1: 0.063
        #     = VALID cF1_thresh 0.167, thr value = 0.838
        # #===========================================================================#

In [ ]:
result = []
for batch in predictions:
    cancer_probs, patient_ids, lateralitys = batch
    cancer_probs = cancer_probs.detach().cpu().numpy()
    for prob, patient_id, laterality in zip(cancer_probs, patient_ids, lateralitys):
        prediction_id = '{}_{}'.format(patient_id, laterality)
        result.append([prediction_id, float(prob)])

In [ ]:
import pandas as pd
THRESHOLD = 0.846

df = pd.DataFrame(result, columns = ['prediction_id', 'cancer'])
sub = df[['prediction_id', 'cancer']].groupby("prediction_id").mean().reset_index()
sub["cancer"] = (sub["cancer"] > THRESHOLD).astype(int)
sub

,prediction_id,cancer
0,10008_L,0
1,10008_R,0
